# Analyzing a Document

We are going to move from an individual sentence to a document. 


## Preamble

This notebook expects that the needed packages have already been installed and that NLTK has been setup correctly via the `setup` notebook.

In [ ]:
import nltk

import re

## Reading in Text

First, lets get some text data. 

Here we use what we've learned about python to read in some text.

In [ ]:
filename = "data/alice.txt"

with open(filename) as handle:
    text = handle.read()

len(text)

In [ ]:
text[0:50]

Real quick, lets take away newline characters to be able to read the text better. 

In [ ]:
# remove new line characters
text = re.sub(r'\n', ' ', text)

text[0:50]

## Tokenization

From our Sentence analysis, we know that the process of splitting up a document into small, word-like _meaningful units_ is known as **tokenization**. 

We will start with word tokenization and look at multi-word tokens in a bit.


In [ ]:
# use word tokenizer
tokens = nltk.word_tokenize(text)

len(tokens)

In [ ]:
tokens[0:20]

Remember, NLTK's word tokenizer leaves punctuation as separate tokens. We will take care of that in a second.

## Searching in a Document



NLTK has some search features in their `Text` object. 

Useful for interactive searching.

In [ ]:
# nltk.Text has useful methods for searching
my_text = nltk.Text(tokens)

# But it initially looks the same as are tokens array
my_text[:20]

**`findall`** takes a string with tokens delimited with angle brackets `<token>`. Regular expressions can be used in and around the angle brackets to robustly find token matches.

Here we find the preceding and following word for every use of "Hare" in the document.

In [ ]:
my_text.findall('<.*><Hare><.*>')

**`concordance`** searches for a word in a document and displays matches in their original contexts. 


Let's create a KWIC for lines that include the word _Hare_

In [ ]:
my_text.concordance('Hare')

A **concordance** is a listing of all the words in a book or other document. Typically the presentation of concordance lines used here is known as a [keyword-in-context](https://en.wikipedia.org/wiki/Key_Word_in_Context) (KWIC) visual.


<h3 style="color:red" class="exercise">Your Turn</h3>

Let's do some more searching. 

In [ ]:
# 1.4.1

## Your code:
# Search for other characters in the text. Where does Hatter show up? What about Alice? 
# Can you use your Regex knowledge to find uppercase and lowercase 'hatter' references?


**Why is this useful?**

A powerful use of keyword-in-context displays comes from a recent Boston Globe piece cataloging [words spoken by suspects at or around their arrest](http://apps.bostonglobe.com/graphics/2016/04/arresting-words/).

Here is a screenshot of the piece showing mentions of the word "phone":

![](imgs/arrest_phones.png)

## Token Counts

Perhaps the most basic, but still insightful metric we could get from an entire document a count of unique tokens.

This gives us a sense of the vocabulary size and repetition of words. 

Let's make a function to count the number of times each token appears in our document and use it on our word tokens. 

Sound fun? Great!

In [ ]:
from collections import Counter

# input: list of tokens
# returns: dict of counts
def get_counts(tokens):
    return Counter(tokens)


# input: dictionary of tokens:counts
# returns: sorted list of (token, count)
def sort_counts(counts):
    return sorted(counts.items(), key=lambda count: count[1], reverse=True)




# get_sorted_counts just runs get_counts and sort_counts together.

# input: list of tokens
# returns: list of (token, count) values 
#  sorted with most used counts on top.
def get_sorted_counts(tokens):
    return sort_counts(get_counts(tokens))

In [ ]:

# how many counts does Alice have?
counts = get_counts(tokens)
counts['Alice']

We can also use `get_sorted_counts` to see most used tokens in our document

In [ ]:

# what are the top used tokens in our data?
sorted_counts = get_sorted_counts(tokens)
sorted_counts[0:20]

# Token Filtering & Transformation

### Removing Punctuation

There is a lot of punctuation in that top list. Let's deal with that now. 

We can create a new function that strips out any tokens that are considered punctuation. 

In [ ]:
# input: list of tokens
# input: string or list of tokens to remove
# output: list of tokens with remove_tokens removed
def remove_tokens(tokens, remove_tokens):
    return [token for token in tokens if token not in remove_tokens]
        

In [ ]:
# we can get a starting point for punctuation from python string
from string import punctuation

# augmenting the base set - to better fit this data.
punc = punctuation + "--''`"
print(punc)

In [ ]:
no_punc_tokens = remove_tokens(tokens, punc)
no_punc_tokens[0:10]

### Normalizing Case

You also notice that some of our words start with a capital letter and some don't. We can normalize all our tokens by using a function to convert them all to lowercase. 

In [ ]:
# input: list of tokens
# output: list of tokens with every token having only lowercase letters.
def lowercase(tokens):
    return [token.lower() for token in tokens]

Lets combine the two to **normalize** our tokens

In [ ]:

normalized_tokens = remove_tokens(lowercase(tokens), punc)

# run sort again
sorted_counts = get_sorted_counts(normalized_tokens)

sorted_counts[0:20]

### Removing Stop Words

We have a list of top words - but they mostly look pretty boring. Of course 'the' is the most used word here - because it is the most used word everywhere! 

The rationale behind doing this is that typically these words add little value towards extracting meaning from text - as they are used so frequently in normal text. So we can just take them out! 

But be careful about this filtering process! With this we are again removing data, so it’s good to just pause and make sure this reduction is useful.


In [ ]:
# import stopwords from nltk
from nltk.corpus import stopwords
stops = stopwords.words('english')

# check out what they look like.
print(len(stops))
print(stops[0:10])

In [ ]:
# use remove_tokens to remove stop words
filtered_normalized_tokens = remove_tokens(normalized_tokens, stops)

sorted_counts = get_sorted_counts(filtered_normalized_tokens)

sorted_counts[0:20]

<h3 style="color:red" class="exercise">Your Turn</h3>

Data analysis is 80% data cleaning right? So let's keep cleaning!

Notice we still have a lot of odd tokens that are the ends of concatenations. 

Develop a function to remove these tokens from our list.

In [ ]:
# 1.4.2


## Your code:
# Finish this function to filter these word fragments
# Hint: "'" not in token  
# ^ this will return true if the ' character isn't in the string 'token'
def remove_word_fragments(tokens):
    return tokens



# Then we will call it to further filter our words.
more_filtered_normalized_tokens = remove_word_fragments(filtered_normalized_tokens)

sorted_counts = get_sorted_counts(more_filtered_normalized_tokens)
sorted_counts[0:20]

## Stemming

Stemming is a transformation process that seeks to convert words to their "base" or root forms. 

So, for example, **housing**, **housed**, and **house** could all be collapsed to the root **hous**. 

The idea is to collapse these similar words into a single token representation in the document. 

This isn't great for visualizations - but can be useful when counting, comparing, or developing other metrics. 

Here we use the `PorterStemmer` which implements one of the most popular stemming algorithms for English-language documents. 

(This algorithm is called the "Porter Stemmer" because it was developed by a Dr. [Martin Porter](https://en.wikipedia.org/wiki/Martin_Porter) in 1980.)


In [ ]:
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

some_words = ['house', 'housing', 'housed', 'mouse', 'mousy', 'mousey']

stemmed_words = [stemmer.stem(word) for word in some_words]
stemmed_words

# Keywords

Extracting keywords is a very important tasks when working with text. Keywords start to answer the question "What is this document about". 

They can work to summarize a document, providing a starting point for topic analysis. Keywords can also show what makes a document unique.

But what are keywords? Keywords (or keyphrases) can be defined as _distinctive_ words or phrases in a document. Obviously, this definition relies on what we mean by distinctive. As you might expect, there are many methods for conjuring up distinctive phrases from text. 

Here we will look at two approaches:

* comparing terms to other terms within the document with collocations.
* comparing words to other words from an external corpus.

## Collocations

A **collocation** is a set of words that occur together more often then chance. These expressions consist of two or more words and can sometimes correspond to some conventional way of saying something.



NLTK has a handy **Collocation Finder** class that can help us here. 

We create a new bigram finder by passing in our tokens to the `BigramCollocationFinder.from_words()` function:

In [ ]:
from nltk.collocations import BigramCollocationFinder

finder = BigramCollocationFinder.from_words(normalized_tokens)

But to get "good" common bigrams out, we need to define a metric to sort bigrams so that interesting collocations can bubble to the top. 

_So what should this sorting metric be?_

NLTK has a few built in ones to choose from, let's look at one. 


### Token Frequency

We have been using word frequency a lot to examine our data processing procedures, what if we just used counts (frequencies) of bigrams? 

More interesting collocations should appear more often - right?

Lets use the `raw_freq` meausure to score bigrams based on counts.

In [ ]:
# built in bigram metrics are in here
bigram_measures = nltk.collocations.BigramAssocMeasures()


# we call score_ngrams on the finder to produce a sorted list
# of bigrams. Each comes with its score from the metric, which
# is how they are sorted. 
finder.score_ngrams(bigram_measures.raw_freq)[:10]

What? These collocations are **BORING**

As you might guess by now, 

It would be better to use a scoring function that took into account the unusual-ness of the terms into account.



-----



### Log likelihood

A better method could use the _likelihood_ of two words occurring together to generate a distance metric. 

Log likelihood is one such method. Log likelihood looks at the counts of events occuring together vs them occuring separately to try to tease out the difference between _surprise_ and _coincidence_. 

Here, the events are any two words appearing together. The calculation looks at how often words appear together vs the same words appearing not together to come up with likelihood scores. 

Fortunately, NLTK makes it super easy to use!

In [ ]:
finder.score_ngrams(bigram_measures.likelihood_ratio)[0:20]

We can see an improvement over raw frequencies in terms of what we would consider 'interesting' phrases.

What if we combined this scoring with stop word removal? 

<h3 style="color:red" class="exercise">Your Turn</h3>

Create a Bigram Collocation Finder and run it on the tokens with stop words filtered out. 


In [ ]:
# 1.4.5

## Your code
# create a BigramCollocationFinder from the stopword filtered tokens
# use your choice of scoring metric to score bigrams. 
# You can use tab completion on the bigram_measures to see other metrics too



**Further Reading**

* [Surprise and Coincidence](http://tdunning.blogspot.com/2008/03/surprise-and-coincidence.html)
* [Stereotropes likelihood](http://stereotropes.bocoup.com/about#gender-association)


# Comparing to the English Language

### Term Frequency - Inverse Document Frequency

We have seen that word counts, also known as **term frequency** is not a very useful metric when attempting to find interesting words or phrases. Even with stopwords removed, the most frequent terms in a document are typically pretty boring. 

The core problem really is that not all words in a document carry the same weight in terms of significance or pertinence. 

The words "the" and "turtle" do not provide the same amount of information. "the" is a word used all the time in english. If a document has a bunch of "the"s in it, that really doesn't tell us anything. But, if you see lots of "turtle"s, you might want to pay attention.
 

TF-IDF tries to capture this idea: _words that don’t occur in often in communication but which occur a lot in your document are important to the document’s content._

While it would be nice to know the frequency of use of all words in all communications in a language, practically speaking - that is impossible.

So a **corpus** or collection of documents is typically used as a proxy to what general communcation looks like. 

We compare our document against a collection of documents to find out what words or phrases make our document unique

---------------

### TF-IDF Calculation

But what is TF-IDF exactly, and how do we calculate it? Let's get to that now!

We know **term frequency** is just the count of a particular token or term in a document. 

**document frequency** is defined as the count of **documents** that a particular token appears in. 


_Quick Example:_


Say we have 3 documents in our corpus, and the token we are looking for is "turtle". 

* Doc 1 has turtle 3 times
* Doc 2 has turtle 0 times
* Doc 3 has turtle 12 times

Those counts right there are the term frequencies for the particular term in particular documents. 

The document frequency of "turtle" would be _2_ - as two of the three of the documents in the corpus contain turtle. 


-----

To get the **inverse document frequency**, we simply take 

```
1 / document frequency
```

So the full calculation is just:

```
term frequency * (1 / document frequency)
```

Seems too simple to be true - and it is!




The calculation for **IDF** we will use is:

```
log(1 + N / document frequency)
```

Where `N` is the number of documents in the corpus.


I said the above calculation was too easy, and it is. There are a few optimizations to IDF that are typically applied.

* We want to make sure we aren't dividing by zero.

* The linear weighting is a bit heavy handed. A rare token found in two documents is most likely not half as interesting as a token found in just one document. 

* We probably want to normalize based on corpus size.


Here are some great graphs from a [good TF-IDF explaination](https://porganized.com/2016/03/09/term-weighting-for-humanists/) that show how different IDF calculations look as a function of document frequency. 

![](imgs/idf-curve1.png)

For the term frequency calculation, **TF**, we will also divide by the number of tokens in the document. 

This will make it so that longer documents are not unfairly boosted by their token counts.


## TF-IDF Code

Enough talking, let's get to coding!



We will be using the **brown** corpus, which is a set of documents broken up into differnet categories. 

(and one of the first [computer readable corpula](http://www.essex.ac.uk/linguistics/external/clmt/w3c/corpus_ling/content/corpora/list/private/brown/brown.html) )


In [ ]:
# Add the brown corpus package to our notebook
from nltk.corpus import brown

# Here we can see the categories used to splitup the articles in the dataset.
brown.categories()

In order to calculate TF-IDF we will need to calculate the document frequency for any token. This means we should keep each document in our corpus separate (as opposed to mashing it all together in one big bag of words) so we know if a token occurs in a particular document. 

Here is one way I found to keep the tokens for each document in the brown corpus in a separate list.

In [ ]:

# our corpus tokens will be an list of lists.
news_tokens = []

# Iterate over the filenames for each document in brown. 
# We are using only those documents in the 'news' category to speed things up.
for filename in brown.fileids(categories='news'):
    # Do our processing on the corpus documents to keep things consistent. 
    bwords = brown.words(fileids=filename)
    processed_news = lowercase(bwords)
    processed_news = remove_tokens(processed_news, punc)
    processed_news = remove_tokens(processed_news, stops)

    news_tokens.append(processed_news)

# this is the number of documents in our corpus
len(news_tokens)

Now we will create functions for each portion of the TF-IDF calculation. 

Lets start with **term frequency**. Assuming we have a dictionary of counts for each token in a document, the calculation becomes simple.

In [ ]:
# input: list of tokens
# returns: dict of counts
def get_counts(tokens):
    return Counter(tokens)

# input token: the token we are looking at
# input counts: token count dictionary for one document
def term_frequency(token, counts):
    '''Calculate term frequency for a particular token in a particular document'''
    return counts[token] / float(len(counts.keys()))


We can test this function a bit with our existing tokens

In [ ]:
counts = get_counts(tokens)



print(term_frequency('the', counts))
print(term_frequency('Alice', counts))
print(term_frequency('walrus', counts))

len(counts.keys())

Next up, let's make a `document_frequency` function. 

Again, Document Frequency refers to the number of documents that contain a particular token. 

We will provide a token and our list of lists that stores our corpus. We want out how many documents in that corpus contain the token.

In [ ]:
# input token: a token to search the corpora for
# input corpus_tokens: list of lists of tokens.
#  One list for each document in the corpora.
# output: number of documents in corpora that contain the token.
def document_frequency(token, corpus_tokens):
    '''Returns number of times a token appears in a set of documents'''
    doc_count = 0
    for tokens in corpus_tokens:
        
        if token in tokens:
            doc_count += 1
            
    return doc_count
        

In [ ]:
# testing document_frequency on some bizness and not so bizness words. 
test_words = ["business", "account", "welcome", "alice", "stillsuit"]
[document_frequency(token, news_tokens) for token in test_words]

Now let's add our optimizations to this metric by creating a `inverse_doc_frequency` function 

In [ ]:
import math

# input: token: token we are analyzing 
# input: corpus_tokens: list of lists of tokens.
#  One list for each document in the corpora.
def inverse_doc_frequency(token, corpus_tokens):
    return math.log(1 +  len(corpus_tokens) / (document_frequency(token, corpus_tokens) + 1))


In [ ]:
for token in ["world", "house", "alice", "hatter"]:
    print(token)
    print(document_frequency(token, news_tokens))
    print(inverse_doc_frequency(token, news_tokens))

**And now the big finish!**





In [ ]:
# input document_tokens: a list of tokens that represent a document
# input corpus_tokens: list of lists of tokens.
#  One list for each document in the corpora.
# output: list of (token, tf-idf) values for each unique token in document_tokens
def tf_idf(document_tokens, corpus_tokens):

    
    # Get our token frequencies for all the unique tokens in our document
    token_counts = get_counts(document_tokens)
    
    # iterate through these tokens and calculate the tf-idf
    tfidfs = {}
    for token in token_counts.keys():
        
        tf = term_frequency(token, token_counts)
        idf = inverse_doc_frequency(token, corpus_tokens)
        
        tfidfs[token] = tf * idf
        
    
    return tfidfs

In [ ]:
# ~~~WARNING~~~
# this takes a while to run!
token_tf_idfs = tf_idf(filtered_normalized_tokens, news_tokens)


In [ ]:
sorted_token_tf_idfs = sort_counts(token_tf_idfs)
sorted_token_tf_idfs[0:20]


## Summarization

**Summarization** is the idea of collapsing a document down to a quick digestable chunk or summary. 


This is especially interesting for more newsy and technical documents where an "abstract-like" summary could be enough for a reader to decide if it is worth reading the document in full. 

A [simple but interesting algorithm]((http://courses.ischool.berkeley.edu/i256/f06/papers/luhn58.pdf) was described in 1957.

Check out this [great interactive explaination](http://www.fastforwardlabs.com/luhn/) of this algorithm.

Here is the basic idea:

* Take a document and remove stop words.
* Pick the top X most frequent words, where X is 5 or so.
* Rank sentences in the document based on how many times these most frequent words appear in the sentence.
* Take the top 3 or 4 sentences as the summary. 

Most of these pieces we have already, we just need to put them together. 

**Further Reading:**

* [Intro to Keyphrase Extraction](http://bdewilde.github.io/blog/2014/09/23/intro-to-automatic-keyphrase-extraction/)
* [Term Weighting for Humanists](https://porganized.com/2016/03/09/term-weighting-for-humanists/)